In [1]:
import numpy as np
import csv
import glob
import pickle
import htmd.ui as ht
import htmd.molecule.voxeldescriptors as vd
from tqdm import *
import os
from sklearn.decomposition import PCA
import bcolz as bc


Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. 
https://dx.doi.org/10.1021/acs.jctc.6b00049
Documentation: http://software.acellera.com/
To update: conda update htmd -c acellera -c psi4

New devel HTMD version (1.12.3 python[3.6,<3.7.0a0,3.5,<3.6.0a0]) is available. You are currently on (1.9.10). Use 'conda update -c acellera htmd' to update to the new version. You might need to update your python version as well if there is no release for your current version.

Deprecation warning: "from htmd import *" will be deprecated. 
To import all HTMD shortcuts please from now on use "from htmd.ui import *"


In [2]:
bc.set_nthreads = 56

In [3]:
def _findDonors(mol, bonds):
    donors = np.zeros(mol.numAtoms, dtype=bool)
    hydrogens = np.where((mol.element == 'HD') | (mol.element == 'HS'))[0]
    for h in hydrogens:
        partners = bonds[bonds[:, 0] == h, 1]
        partners = np.hstack((partners, bonds[bonds[:, 1] == h, 0]))
        for p in partners:
            if mol.name[p][0] == 'N' or mol.name[p][0] == 'O':
                donors[p] = True
    return donors

In [4]:
def get_all_channels(mol):
    from collections import OrderedDict
    props = OrderedDict()
    _order = ('p_hydrophobic', 'p_aromatic', 'p_hbond_acceptor', 'p_hbond_donor',
              'p_positive_ionizable', 'p_negative_ionizable', 'p_metal', 'p_occupancies',
              'l_hydrophobic', 'l_aromatic', 'l_hbond_acceptor', 'l_hbond_donor',
              'l_positive_ionizable', 'l_negative_ionizable', 'l_metal', 'l_occupancies')
    elements = np.array([el.upper() for el in mol.element])

    # Proteins
    _map = (elements == 'C') | (elements == 'A')
    props['p_hydrophobic'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('protein'))])
    
    _map = elements == 'A'
    props['p_aromatic'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('protein'))])
    
    _map = (elements == 'NA') | (elements == 'NS') | (elements == 'OA') | (elements == 'OS') | (
    elements == 'SA')    
    props['p_hbond_acceptor'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('protein'))])
    
    _map = _findDonors(mol, mol._getBonds())
    props['p_hbond_donor'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('protein'))])
    
    _map = mol.charge > 0
    props['p_positive_ionizable'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('protein'))])
    
    _map = mol.charge < 0
    props['p_negative_ionizable'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('protein'))])
    
    _map = (elements == 'MG') | (elements == 'ZN') | (elements == 'MN') | \
                     (elements == 'CA') | (elements == 'FE')
    props['p_metal'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('protein'))])
    
    _map = (elements != 'H') & (elements != 'HS') & (elements != 'HD')
    props['p_occupancies'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('protein'))])
    
    # Ligands
    _map = (elements == 'C') | (elements == 'A')
    props['l_hydrophobic'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('not protein'))])
    
    _map = elements == 'A'
    props['l_aromatic'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('not protein'))])
    
    _map = (elements == 'NA') | (elements == 'NS') | (elements == 'OA') | (elements == 'OS') | (
    elements == 'SA')    
    props['l_hbond_acceptor'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('not protein'))])
    
    _map = _findDonors(mol, mol._getBonds())
    props['l_hbond_donor'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('not protein'))])
    
    _map = mol.charge > 0
    props['l_positive_ionizable'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('not protein'))])
    
    _map = mol.charge < 0
    props['l_negative_ionizable'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('not protein'))])
    
    _map = (elements == 'MG') | (elements == 'ZN') | (elements == 'MN') | \
                     (elements == 'CA') | (elements == 'FE')
    props['l_metal'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('not protein'))])
    
    _map = (elements != 'H') & (elements != 'HS') & (elements != 'HD')
    props['l_occupancies'] = np.array([i and j for (i, j) in zip(_map, mol.atomselect('not protein'))])
    

    channels = np.zeros((len(elements), len(props)), dtype=bool)
    for i, p in enumerate(_order):
        channels[:, i] = props[p]
    return channels

In [5]:
# Read the data file and get all the pdb ids
def read_score():
    pdb_ids = []
    scores = {}
    with open('pdbbind_refined_set.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None) # Skip the header
        for row in reader:
            pdb_ids.append(row[1])
            scores[row[1]]= float(row[5])

    return pdb_ids, scores

In [6]:
def _reshape(x):
    new_shape = [50, 50, 50, 16]
    dim_diff = np.array([i-j for i, j in zip(new_shape, x.shape)])
    pad_dim = np.round(dim_diff / 2).astype(int)
    x = np.pad(x, [(pad_dim[0], dim_diff[0]-pad_dim[0]),
                   (pad_dim[1], dim_diff[1]-pad_dim[1]),
                   (pad_dim[2], dim_diff[2]-pad_dim[2]),
                   (0, 0)],
               'constant')
    
#     return x
    return x[13:37, 13:37, 13:37, :]

In [7]:
exp_pdb_ids, scores = read_score()
len(exp_pdb_ids)

4154

In [8]:
data_dir = "../../pdbbind_data/"
files = glob.glob(data_dir + "*/*/*_pocket.pdbqt", recursive=True)
len(files)

4057

In [9]:
valid_files = []
for file in files:
    _id = os.path.split(file)[0][-4:]
    if _id in exp_pdb_ids:
        valid_files.append(file)
len(valid_files)

3794

In [10]:
def transform_coordinates(molecule, verbose=False):
    """
    Method for reorientation of the protein-ligand complex.
    INPUT: Molecule object
    OUTPUT: Molecule object with reoriented coordinates.
        New axes are:
            x-axis = Along the first principal component of the ligand coordinates
            y-axis = Centroid vector of the protein which is orthogonalized to the x-axis.
            z-axis = Orthogonal to both x and y axis.
    """
    # Select protein and ligands
    if verbose: print("INFO: SELECTING PROTEINS AND LIGANDS", end='...')
    p_map = molecule.atomselect('protein')
    l_map = molecule.atomselect('not protein')
    if verbose: print("DONE")
    if verbose: print("INFO: RESHAPING COORDINATES", end='...')
    p_points = molecule.coords[p_map].reshape(-1, 3)
    l_points = molecule.coords[l_map].reshape(-1, 3)
    if verbose: print("DONE")

    #Find the centroid of the ligand structure and move the points around that.
    if verbose: print("INFO: FINDING THE CENTER OF THE LIGAND AND CENTERING", end='...')
    l_centroid = np.mean(l_points, axis=0)
    l_points = l_points - l_centroid
    p_points = p_points - l_centroid
    if verbose: print("DONE")
    
    # Perform PCA on ligand points
    if verbose: print("INFO: PERFORMING PCA", end='...')
    pca = PCA(n_components=3)
    pca.fit(l_points)
    if verbose: print("DONE")
    
    # x_axis is the first principal component
    x_axis = pca.components_[:, 0]
    
    # Get the centroid of the protein points
    p_centroid = np.mean(p_points, axis=0)
    
    # Projection of p_centroid vector on x_axis
    if verbose: print("INFO: PROJECTING THE CENTROID OF THE PROTEIN TO THE FIRST PRINCIPAL AXIS", end="...")
    proj_x = np.matmul(np.transpose(x_axis), p_centroid)/np.matmul(np.transpose(x_axis), x_axis) * x_axis
    # Get y-axis
    y_axis = p_centroid - proj_x
    if verbose: print("DONE")

    #Normalize
    if verbose: print("INFO: NORMALIZING Y-AXIS", end="...")
    y_axis = y_axis / np.linalg.norm(y_axis)
    if verbose: print("DONE")
    
    # z_axis is perpendicular to both x_axis and y_axis. Not sure about the direction yet (+ or - ?)
    z_axis = np.cross(x_axis, y_axis)
    
    # Transformation matrix (as column vector and normalized)
    if verbose: print("INFO: GETTING THE TRANSFORMATION MATRIX", end="...")
    R = np.transpose(np.array([x_axis, y_axis, z_axis]))
    if verbose: print("DONE")
    
    # Transform all protein and ligand points
    if verbose: print("INFO: TRANSORFMING THE COORDINATES OF LIGAND AND PROTEIN", end="...")
    mol_coords = molecule.coords.reshape((-1, 3))
    transformed_coords = np.transpose(np.matmul(R, np.transpose(mol_coords))).reshape((-1, 3, 1))
    #molecule.coords = transformed_coords    
    if verbose: print("DONE")
    
#     p_points = np.transpose(np.matmul(R, np.transpose(p_points))).reshape(-1, 3, 1)
#     l_points = np.transpose(np.matmul(R, np.transpose(l_points))).reshape(-1, 3, 1)
    
#     molecule.coords[p_map] = p_points
#     molecule.coords[l_map] = l_points
    
    if verbose: print("INFO: COORDINATE TRANSFORMATION FINISHED")
    return transformed_coords

In [11]:
def get_features(files, reorient=False, verbose=False):
    for file in files:
        if verbose: print("INFO: GETTING THE MOLECULE OBJECT", end="")
        mol = ht.Molecule(file)
        _id = mol.viewname[:4]
        if verbose: print(" FOR ID: ", _id, "...DONE")
        mol.renumberResidues()
        if not reorient: 
            if verbose: print("INFO: CENTERING THE MOLECULE", end="...")
            c = np.mean(mol.coords, axis=0)
            mol.moveBy(-c) # Coordinate transformation does the centering
            if verbose: print("DONE")
        
        if reorient:
            if verbose: print("INFO: STARTING COORDINATE TRANSFORMATION")
            try:
                mol = transform_coordinates(mol, verbose=True)
                if verbose: print("INFO: COORDINATE TRANSFORMATION DONE")
            except:
                if verbose: print("INFO: COORDINATE TRANSFORMATION FAILED!")
                continue
        try:
            # Get the voxeldescriptors
            if verbose: print("INFO: GETTING THE VOXEL DESCRIPTORS", end="...")
            f, centers, natoms = vd.getVoxelDescriptors(mol, 
                                                        channels=get_all_channels(mol))#, method='CUDA')
            if verbose: print("RESHAPING...", end="...")
            f = f.reshape(natoms[0], natoms[1], natoms[2], -1)
            if verbose: print("DONE")
        except:
            if verbose: print("FAILED")
            continue
        
        if verbose: print("INFO: RESHAPING THE FEATURES AND RETURN", end ="...")
        f = f.reshape(natoms[0], natoms[1], natoms[2], -1)
        if verbose: print("DONE")
            
        if verbose: print("INFO: FEATURES EXTRACTION FINISHED FOR ", _id)
        
        yield _reshape(f), scores[_id]

In [12]:
# Get the Molecule object of an example file
mol = ht.Molecule("../../pdbbind_data/refined-set-2016/4b2i/4b2i_pocket.pdbqt")

In [13]:
# Extract the features
f, centers, natoms = vd.getVoxelDescriptors(mol, channels=get_all_channels(mol))
f1 = f.reshape(natoms[0], natoms[1], natoms[2], -1)

In [14]:
f1.shape

(20, 26, 26, 16)

In [15]:
np.var(f1.reshape((-1, 16)), axis=0)

array([0.11640091, 0.01525822, 0.04217882, 0.03735579, 0.11879581,
       0.07506089, 0.        , 0.13661028, 0.        , 0.        ,
       0.01378874, 0.        , 0.        , 0.        , 0.        ,
       0.01378874])

In [16]:
# Transform the coordinates and push them back to the Molecule object
mol.coords = transform_coordinates(mol)

In [17]:
# Extract the features again
f, centers, natoms = vd.getVoxelDescriptors(mol, channels=get_all_channels(mol))
f2 = f.reshape(natoms[0], natoms[1], natoms[2], -1)

In [18]:
f2.shape

(26, 27, 20, 16)

Shapes are changed now!

In [19]:
np.var(f2.reshape((-1, 16)), axis=0)

array([4.68454850e-04, 0.00000000e+00, 5.98114087e-11, 3.24354878e-14,
       4.68454850e-04, 6.72461220e-11, 0.00000000e+00, 4.68454842e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [20]:
np.var(f1), np.var(f2)

(0.04003784700200238, 8.7889686758965e-05)

Variances are not the same!

In [21]:
# # With coordinate transformation
# x, y = next(get_features(valid_files[:1], reorient=True, verbose=True))
# data_x = bc.carray(x.reshape((1,)+x.shape), rootdir='data_x_transformed')
# data_y = np.array(y)
# count = 0

# pbar = tqdm_notebook(total = len(valid_files[1:]))
# for x, y in get_features(valid_files[1:], reorient=True, verbose=True):
#     print(count, end=',')
#     count = count + 1
#     data_x.append(x.reshape((1,)+x.shape))
#     data_y = np.append(data_y, np.array(y))
#     pbar.update()

# data_x.flush()

In [22]:
# # Without coordinate transformation

# x, y = next(get_features(valid_files[:1]))
# data_x = bc.carray(x.reshape((1,)+x.shape), rootdir='data_x_50')
# data_y = np.array(y)

# pbar = tqdm_notebook(total = len(valid_files[1:]))
# for x, y in get_features(valid_files[1:]):
#     data_x.append(x.reshape((1,)+x.shape))
#     data_y = np.append(data_y, np.array(y))
#     pbar.update()

# data_x.flush()